In [ ]:
%pip install wandb -Uqq

In [ ]:
%pip install nmslib -Uqq

     |████████████████████████████████| 13.5 MB 6.6 MB/s 
     |████████████████████████████████| 188 kB 50.7 MB/s 


In [ ]:
%pip install LibRecommender -Uqq

     |████████████████████████████████| 2.0 MB 6.4 MB/s 
     |████████████████████████████████| 24.1 MB 2.2 MB/s 


In [ ]:
%pip install faiss==1.5.3 -Uqq

     |████████████████████████████████| 4.7 MB 7.4 MB/s 


In [ ]:
%pip install ujson redis -Uqq

     |████████████████████████████████| 45 kB 2.5 MB/s 
     |████████████████████████████████| 246 kB 14.9 MB/s 


In [ ]:
import wandb
wandb.login()

In [ ]:
import os
from pathlib import Path

import numpy as np
import pandas as pd

from libreco.data import random_split, split_by_ratio_chrono, DatasetPure
from libreco.algorithms import Item2Vec   # pure data
from libreco.evaluation import evaluate

import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

In [ ]:
from libserving.serialization import save_embed

In [ ]:
MODEL_PATH = "/content/drive/MyDrive/Colab Notebooks/00-MTC/models/Item2Vec" 
if not os.path.isdir(MODEL_PATH):
    os.mkdir(MODEL_PATH)  
MODEL_NAME = 'Item2Vec'

In [ ]:
data = pd.read_csv("https://s3-us-west-2.amazonaws.com/recommender-tutorial/ratings.csv")
data.columns = ["user", "item", "label", "time"]
data["label"] = 1 

train_data, eval_data, test_data = split_by_ratio_chrono(data, multi_ratios=[0.8, 0.1, 0.1])

train_data, data_info = DatasetPure.build_trainset(train_data)
eval_data = DatasetPure.build_evalset(eval_data)
test_data = DatasetPure.build_testset(test_data)
train_data.build_negative_samples(data_info)  # sample negative items for each record
eval_data.build_negative_samples(data_info) 
test_data.build_negative_samples(data_info)
print(data_info)   # n_users: 5894, n_items: 3253, data sparsity: 0.4172 %

random neg item sampling elapsed: 0.099s
random neg item sampling elapsed: 0.011s
random neg item sampling elapsed: 0.011s
n_users: 610, n_items: 8237, data sparsity: 1.6056 %


In [ ]:
window_size = max(len(seq) for seq in data_info.user_consumed.values()) + 5
print('max window_size:', window_size)

max window_size: 2163


In [ ]:
window_size_2 = window_size // 2
window_size_2

1081

In [ ]:
window_size_3 = window_size // 3
window_size_3

721

In [ ]:
window_size_4 = window_size // 4
window_size_4

540

In [ ]:
MODEL_PATH = "/content/drive/MyDrive/Colab Notebooks/00-MTC/models/Item2Vec" 
MODEL_NAME = 'Item2Vec'
cfg = {
  "embed_size": 16,
  "n_epochs": 3,
  "norm_embed": False,
  "window_size": None,
  "k":10,
}
sweep_config = {
    "name": "Item2Vec-sweep",
    "metric": {"name": "loss", "goal": "minimize"},    
    "method": "random",  
    'early_terminate':{
      'type': 'hyperband',
      'min_iter': 10
  },  
    "parameters": {        
          "embed_size": {
              "values": [16, 32, 64]
          },
          'n_epochs': {
              "values": [5, 10, 20]
          }, 
          "norm_embed": {
              "values": [False, True]
          },
                    
          "window_size": {
              "values": [2160, 1080, 540]
          },
          }
}
metrics = ["loss", "roc_auc", "precision", "recall", "map", "ndcg"]    
sweep_id = wandb.sweep(sweep_config, 
                       project="recsys-app", 
                       entity="iloncka-ds")

os.environ["WANDB_RUN_GROUP"] = "experiment-Item2Vec"

def train_with_wandb():  
      with wandb.init(group=MODEL_NAME,
                      notes=f"{MODEL_NAME} hyperparameters search",
                      tags=["embed", "sequence", "hyperparameters"]) as runwb:
        # Overwrite the random run names chosen by wandb
        sc = runwb.config
        name_str = f'{MODEL_NAME}_met_{sweep_config["method"]}' + \
          f'_n_epoch_{sc.n_epochs}_' + \
          f'emb_size_{sc.embed_size}_norm_embed_{sc.norm_embed}_' + \
          f'window_size_{sc.window_size}'
        runwb.config.update({"name": name_str})
        current_dir = os.path.join(MODEL_PATH, name_str)
        if not os.path.isdir(current_dir):
            os.mkdir(current_dir)  
        # Rest of code.
        model = Item2Vec(task="ranking", data_info=data_info, 
                        embed_size=sc.embed_size, n_epochs=sc.n_epochs, 
                        norm_embed=sc.norm_embed, window_size=sc.window_size)
        #NB!! param runwb is absent in official version of LibRecommender, for training unpublished changes to this library were used
        model.fit(runwb, train_data, eval_data=eval_data, metrics=metrics)
        #NB!! param runwb is absent in official version of LibRecommender, for training unpublished changes to this library were used
        evaluate(runwb=runwb, model=model, data=test_data,
                                    metrics=metrics)
        # specify model saving directory
        save_embed(current_dir, model)        
        model.save(path=current_dir, model_name="Item2Vec", manual=True, inference_only=True)
wandb.agent(sweep_id, function=train_with_wandb, entity="iloncka-ds", project="recsys-app", count=30)

Create sweep with ID: cmrz31in
Sweep URL: https://wandb.ai/iloncka-ds/recsys-app/sweeps/cmrz31in


wandb: Agent Starting Run: 3vxtwqr5 with config:
wandb: 	embed_size: 64
wandb: 	n_epochs: 20
wandb: 	norm_embed: False
wandb: 	window_size: 2160
ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: iloncka-ds. Use `wandb login --relogin` to force relogin


window size: 2160, using too large window size may slow down training.
Training start time: 2022-11-07 07:39:40


Item2vec iter20: 100%|██████████| 610/610 [00:04<00:00, 123.60it/s]


gensim word2vec training elapsed: 542.510s


...

window size: 1080, using too large window size may slow down training.
Training start time: 2022-11-07 09:45:29


Item2vec iter20: 100%|██████████| 610/610 [00:04<00:00, 139.68it/s]


gensim word2vec training elapsed: 428.168s


eval_rec: 100%|██████████| 608/608 [00:00<00:00, 1299.83it/s]


loss,▁
map,▁
ndcg,▁
precision,▁
recall,▁
roc_auc,▁
loss,1.185
map,0.00238
ndcg,0.0045
precision,0.00132
recall,0.00073


https://wandb.ai/iloncka-ds/recsys-app/groups/Item2Vec/workspace?workspace=user-iloncka-ds